In [37]:
import os 
import numpy as np 

label_path = './dataset-split/labels/'
pedestrian_label = 3

img_folder_path = 'D:/Projects/SEAA-2021/subset/images/' 
label_folder_path = 'D:/Projects/SEAA-2021/subset/labels/' 


In [38]:
box_heads = './occluded_boxes/'
wall_legs = './occluded_walls/'

if not os.path.isdir(box_heads):
    os.mkdir(box_heads)
    os.mkdir(box_heads + 'images')
    os.mkdir(box_heads + 'labels')
    
if not os.path.isdir(wall_legs):
    os.mkdir(wall_legs)
    os.mkdir(wall_legs + 'images')
    os.mkdir(wall_legs + 'labels')

In [39]:
import numpy as np
import cv2 as cv
import sys
import os

def overlay_image_alpha(img, img_overlay, pos, alpha_mask):
    """Overlay img_overlay on top of img at the position specified by
    pos and blend using alpha_mask.
    Alpha mask must contain values within the range [0, 1] and be the
    same size as img_overlay.
    """
    x, y = pos

    # Image ranges
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Overlay ranges
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Exit if nothing to do
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    channels = img.shape[2]

    alpha = alpha_mask[y1o:y2o, x1o:x2o]
    alpha_inv = 1.0 - alpha

    for c in range(channels):
        img[y1:y2, x1:x2, c] = (alpha * img_overlay[y1o:y2o, x1o:x2o, c] +
                                alpha_inv * img[y1:y2, x1:x2, c])


def main(fn_occ, fn_kitti, fn_label, out_folder,upper=False):
    input_kitti = os.path.basename(fn_kitti)
    box = cv.imread(fn_occ, -1)
    width_box = box.shape[1]
    height_box = box.shape[0]

    img = cv.imread(fn_kitti, -1)
    width = img.shape[1]
    height = img.shape[0]

    with open(fn_label) as f:
        content = f.readlines()
    content = [x.strip() for x in content]

    if upper: 
        scale = 1.0
    else: 
        scale = 1.3
    
    # label all pedestrians
    for lbl in content:
        entry = lbl.split(' ')
        if int(entry[0]) == 3:
            # pedestrian
            c_x = int(float(entry[1]) * float(width))
            c_y = int(float(entry[2]) * float(height))
            w = int(float(entry[3]) * float(width))
            h = int(float(entry[4]) * float(height))
            
            if upper: 
                tl_x = int(c_x - w/2)
                tl_y = int(c_y - h/2)
            else: 
                tl_x = int(c_x - 2*w/3)
                tl_y = int(c_y + .2*h/2)
                
            #br_x = int(c_x + w/2)
            #br_y = int(c_y + h/2)
            #cv.rectangle(img, (tl_x, tl_y), (br_x, br_y), (0, 0, 255), 2)
            scaled_box = cv.resize(box, None, fx=scale*w/width_box, fy=scale*w/width_box)
            overlay_image_alpha(img, scaled_box[:, :, 0:3], (tl_x, tl_y), scaled_box[:, :, 3] / 255.0)

    cv.imwrite(out_folder + '/' + input_kitti, img)
    #cv.imshow('view', img)
    #ch = 0xFF & cv.waitKey()

In [40]:
images = os.listdir(img_folder_path)
#output_path = box_heads + 'images/'
#overlay_image = "./overlay images/box.png"

output_path = wall_legs + 'images/'
overlay_image = "./overlay images/room46.png"


for f in images: 
    kitti_image = img_folder_path + f
    label_path = label_folder_path + f.split('.')[0] + '.txt'

    # Boxheads 
    #main(overlay_image, kitti_image, label_path, output_path,upper=True)

    # Wall legs 
    main(overlay_image, kitti_image, label_path, output_path,upper=False)

#main(str(sys.argv[1]), str(sys.argv[2]), str(sys.argv[3]), str(sys.argv[4]))
#cv.destroyAllWindows()